In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import astropy
from astropy.table import Table 
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from importlib import reload

import sys
sys.path.insert(0,'/home/imendoza/alcca/nbody-relaxed/intro') #where my code is. 

In [ ]:
#my packages
import plotting
reload(plotting);

from plotting import to_latex, scatter_binning, binning3d_mass


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.rc("text", usetex=True)

# preamble 

In [ ]:
#functions I need to define myself. 
cvir = lambda cat: cat['rvir'] / cat['rs']
phi_l = lambda cat: np.arccos(
    ((cat['A[x]']*cat['Jx'] + cat['A[y]']*cat['Jy'] + cat['A[z]']*cat['Jz'])
     /
    (np.sqrt(cat['A[x]']**2 + cat['A[y]']**2 + cat['A[z]']**2)*np.sqrt(cat['Jx']**2 + cat['Jy']**2 + cat['Jz']**2) )
    )
)
q = lambda cat: (1/2)*(cat['b_to_a'] + cat['c_to_a']);

In [ ]:
#information about catalogue (https://www.cosmosim.org/cms/simulations/bolshoi/)
particle_mass = 1.35e8 #Msun
total_particles = 2048**3 
box_size = 250 #Mpc/h

# read

In [ ]:
from astropy.io import ascii

In [ ]:
filename = '/home/imendoza/alcca/nbody-relaxed/intro/data/sample_data/hlist_1.00109.csv'

In [ ]:
#100 Mb chunks of maximum memory in each iteration. 
#this returns a generator. 
tbls = ascii.read(filename, format='csv', guess=False,
                 fast_reader={'chunk_size': 100 * 1000000, 'chunk_generator': True})

In [ ]:
ftbls = [] 
for i, tbl in enumerate(tbls): 
    tbl.add_column(cvir(tbl), name='cvir')
    tbl.add_column(phi_l(tbl), name='phi_l')
    tbl.add_column(q(tbl), name='q')
    
    ftbls.append(tbl[plotting.params])
    if i%10 ==0: 
        print(i)

ftbl = astropy.table.vstack(ftbls)

In [ ]:
#only look at things that have at least 1000 particles as several authors suggest. 
#others might be too noisy. 
cat = ftbl[ (np.log10(ftbl['mvir']) > 12) & 
          (ftbl['Spin'] != 0) &
          (ftbl['q'] !=0)]

cat['Xoff'] = cat['Xoff'] / cat['rvir'] #normalize to definition used in Power2011 (and others)

In [ ]:
#following recommendations from power2011dynamical
#eta = 2 * T/U 
relaxed = (cat['Xoff'] < 0.04)

In [ ]:
len(cat[relaxed])

# histograms

In [ ]:
#mass
plt.figure(figsize=(8,8))
plt.hist(np.log10(cat['mvir']), bins=30, histtype='step', label='all', color='r');
plt.hist(np.log10(cat[relaxed]['mvir']), bins=30, histtype='step',color='b', label='relaxed');
plt.xlabel(to_latex('mvir', True, True), size=20)

plt.yscale('log')
plt.legend(prop={'size':18})

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)


In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(10,20))
params = ['mvir', 'cvir' , 'T/|U|', 'Xoff', 'Voff', 'Spin', 'q', 'phi_l']
# print('mean, median, and std of:')
for param, ax in zip(params, axs.flatten()): 
    
    #all halos 
    x = np.log10(cat[param]) 
    px = (x - np.mean(x))/ np.std(x)
    ax.hist(px, histtype='step', bins=30, label='all', color='r')
    
    #relaxed halos 
    x = np.log10(cat[relaxed][param]) 
    px = (x - np.mean(x))/ np.std(x)
    ax.hist(px, histtype='step', bins=30, label='relaxed', color='b')
    ax.set_xlabel(to_latex(param,True), fontsize=18)
    ax.tick_params('both', labelsize=18)
    ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    ax.yaxis.get_offset_text().set_fontsize(18)

#     print(f'{param}:', np.mean(x), np.median(x), np.std(x))
    
    ax.legend(loc='best', prop={'size':20})

fig.tight_layout() 

# Vanilla Scatters and Contours

In [ ]:
#all masses in catalogue
plt.scatter(cat['mvir'], cat['Xoff'], alpha=0.1)
plt.xscale('log')
plt.yscale('log')
print("correlation:", spearmanr(cat['mvir'], cat['Xoff'])[0])

In [ ]:
from astroML.plotting import scatter_contour
scatter_contour(np.log10(cat['mvir']), np.log10(cat['Xoff']), filled_contour=True, threshold=100, levels=10)

# correlations

## matrix version

In [ ]:
params = ['mvir', 'cvir' , 'T/|U|', 'Xoff', 'Voff', 'Spin', 'q', 'phi_l']
latex_params= [to_latex(param) for param in params]

In [ ]:
#prettier version. 
corrs = np.zeros((len(params), len(params)))
for i,param1 in enumerate(params): 
    for j,param2 in enumerate(params):
        corr, p = spearmanr(cat[param1], cat[param2])
        corrs[i,j] = corr

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
mask =  np.tri(corrs.shape[0], k=-1)
A = np.ma.array(corrs, mask=mask)
im = ax.matshow(A,cmap='bwr',vmin=-1, vmax=1)
plt.colorbar(im, ax = ax)
ax.set_xticklabels(['']+latex_params, size=16);
ax.set_yticklabels(['']+latex_params, size=16);

In [ ]:
#print by hand. 
np.set_printoptions(threshold=100)
print(params)
corrs

## Correlations between parameters, graphically

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(18,10))
axes = axs.flatten()


scatter_binning(np.log10(cat['mvir']), np.log10(cat['T/|U|']), nxbins=10, ax=axes[0],
               xlabel=to_latex('mvir', True, True), ylabel=to_latex('T/|U|',True), xlabel_size=24, ylabel_size=24,
               tick_size=18, no_bars=True, show_bands=True, legend_label='all galaxies')

scatter_binning(np.log10(cat[relaxed]['mvir']), np.log10(cat[relaxed]['T/|U|']), nxbins=10, ax=axes[0],
               xlabel=to_latex('mvir', True, True), ylabel=to_latex('T/|U|',True), xlabel_size=24, ylabel_size=24,
               tick_size=18, no_bars=True, show_bands=True, legend_label='relaxed', legend_size=18, color='blue'
               )

scatter_binning(np.log10(cat['mvir']), np.log10(cat['Xoff']), nxbins=10, ax=axes[1], 
                xlabel=to_latex('mvir', True,True), ylabel=to_latex('Xoff',True), xlabel_size=24, ylabel_size=24,
               tick_size=18, no_bars=True, show_bands=True, legend_label='all galaxies')

scatter_binning(np.log10(cat[relaxed]['mvir']), np.log10(cat[relaxed]['Xoff']), nxbins=10, ax=axes[1], 
                xlabel=to_latex('mvir', True,True), ylabel=to_latex('Xoff',True), xlabel_size=24, ylabel_size=24,
               tick_size=18, no_bars=True, show_bands=True, legend_label='relaxed', legend_size=18, color='blue')

scatter_binning(np.log10(cat['mvir']), np.log10(cat['Voff']), nxbins=10, ax=axes[2], 
                xlabel=to_latex('mvir', True,True), ylabel=to_latex('Voff',True), xlabel_size=24, ylabel_size=24,
                tick_size=18, no_bars=True, show_bands=True, legend_label='all galaxies')

scatter_binning(np.log10(cat[relaxed]['mvir']), np.log10(cat[relaxed]['Voff']), nxbins=10, ax=axes[2], 
                xlabel=to_latex('mvir', True,True), ylabel=to_latex('Voff',True), xlabel_size=24, ylabel_size=24,
               tick_size=18, no_bars=True, show_bands=True, legend_label='relaxed', legend_size=18, color='blue')

plt.tight_layout()

In [ ]:
fig, axs=  plt.subplots(3,2, figsize=(16,18))
axes = axs.flatten()
plotting.binning3d_mass(cat, axes[0], scatter_binning, 'Xoff', 'T/|U|', mods=[lambda x: np.log10(x), lambda x: np.log10(x)],
               plot_kwargs=dict(nxbins=10, xlabel=plotting.to_latex('Xoff',True), ylabel=plotting.to_latex('T/|U|'), 
                             tick_size=22, xlabel_size=24, ylabel_size=24), legend_size=22
              )
plotting.scatter_binning(np.log10(cat['Xoff']), np.log10(cat['T/|U|']), axes[1], 
                         xlabel=plotting.to_latex('Xoff',True), ylabel=plotting.to_latex('T/|U|'),
                        tick_size=22, xlabel_size=24, ylabel_size=24)

plotting.binning3d_mass(cat, axes[2], scatter_binning, 'Xoff', 'Voff', mods=[lambda x: np.log10(x), lambda x: np.log10(x)],
               plot_kwargs=dict(nxbins=10, xlabel=plotting.to_latex('Xoff',True), ylabel=plotting.to_latex('Voff',True), 
                             tick_size=22, xlabel_size=24, ylabel_size=24), legend_size=22
              )

plotting.scatter_binning(np.log10(cat['Xoff']), np.log10(cat['Voff']), axes[3], xlabel=plotting.to_latex('Xoff',True), 
                         ylabel=plotting.to_latex('Voff'), tick_size=22, xlabel_size=24, ylabel_size=24)

plotting.binning3d_mass(cat, axes[4], scatter_binning, 'Voff', 'T/|U|', mods=[lambda x: np.log10(x), lambda x: np.log10(x)],
               plot_kwargs=dict(nxbins=10, xlabel=plotting.to_latex('Voff',True), ylabel=plotting.to_latex('T/|U|',True), 
                             tick_size=22, xlabel_size=24, ylabel_size=24), legend_size=22
              )

plotting.scatter_binning(np.log10(cat['Voff']), np.log10(cat['T/|U|']), axes[5], xlabel=plotting.to_latex('Voff',True), 
                         ylabel=plotting.to_latex('T/|U|'), tick_size=22, xlabel_size=24, ylabel_size=24)
plt.tight_layout()
# scatter_binning(np.log10(cat[]))

## Investigate weird correlation between Xoff and Mvir 

In [ ]:
fig, ax =plt.subplots(1,1,figsize=(12,12))
scatter_binning(np.log10(cat['mvir']), np.log10(cat['Xoff']), nxbins=10, ax=ax 
                xlabel=to_latex('mvir', True,True), ylabel=to_latex('Xoff',True), xlabel_size=24, ylabel_size=24,
               tick_size=18, no_bars=True, show_bands=False, legend_label='all galaxies')

# PCA 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
params = ['mvir', 'cvir','Spin', 'q','phi_l', 'Xoff', 'T/|U|'
         ]

In [ ]:
pca = PCA(n_components=4)

In [ ]:
#normalize and take log.
#cuts on mass >= 12 once we get z= 0 catalogue and different fixed mass regimes too.  
# remove extreme outliers too. 
X = np.zeros((len(params), len(cat['mvir'])))
for i, param in enumerate(params): 
    x = np.log10(cat[param])
    px = (x - np.mean(x))/ np.std(x)
    X[i] = px 
    

In [ ]:
pca.fit(X.T)

In [ ]:
np.set_printoptions(threshold=50)
print(params)
print('singular values:', pca.singular_values_)
print('explained variance ratio:', pca.explained_variance_ratio_)  
print('components \n ', pca.components_)

# dynamical comparisons 

Useful to compare to the Power et al. 2011 paper

In [ ]:
params = ['mvir', 'cvir' , 'T/|U|', 'Xoff', 'Voff', 'Spin', 'q', 'phi_l', 'Acc_Rate_Inst',
          'Acc_Rate_1*Tdyn','Acc_Rate_2*Tdyn']

In [ ]:
#not sure what Halfmass_Scale corresponds to but maybe inverse of (1+zform)^(-1)? 
#sanity check. 
fig, ax = plt.subplots(1,1,figsize=(8,8))
scatter_binning(np.log10(cat['mvir']), cat['cvir'], ax, xlabel=to_latex('mvir',True), ylabel=to_latex('cvir'),
               xlabel_size=22, ylabel_size=28, legend_label='all', no_bars=True, show_bands=True)

scatter_binning(np.log10(cat[relaxed]['mvir']), cat[relaxed]['cvir'], ax, xlabel=to_latex('mvir',True), ylabel=to_latex('cvir'),
               xlabel_size=22, ylabel_size=28, color='blue', legend_label='relaxed', no_bars=True, show_bands=True)

ax.legend(prop={'size':18}, loc='best')

In [ ]:
#accretion rate and mass. 
fig, axs = plt.subplots(1,2,figsize=(20,10))

scatter_binning(np.log10(cat['mvir']), cat['Acc_Rate_1*Tdyn'],ax=axs[0], xlabel=to_latex('mvir',True), 
                ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, ylabel_size=32, tick_size=22, no_bars=True, 
               show_lines=True)
axs[0].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
axs[0].yaxis.get_offset_text().set_fontsize(24)
axs[0].set_yscale('log',nonposy='clip')

# axs[0].set_yscale('log')

scatter_binning(np.log10(cat['mvir']), cat['Acc_Rate_Inst'],ax=axs[1], xlabel=to_latex('mvir',True), 
                ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, ylabel_size=32, tick_size=22, 
               no_bars=True, show_lines=True)
axs[1].ticklabel_format(style='sci', axis='y', scilimits=(0,0))
axs[1].yaxis.get_offset_text().set_fontsize(24)
axs[1].set_yscale('log',nonposy='clip')
# scatter_binning(np.log10(cat['mvir']), np.log10(cat['Acc_Rate_1*Tdyn']) )

In [ ]:
fig, axs = plt.subplots(6,2,figsize=(18,30))
axes = axs.flatten()
binning3d_mass(cat, axes[0], scatter_binning, 'Xoff', 'Acc_Rate_1*Tdyn', 
                   mods=[lambda x: np.log10(x), lambda x: x],
                   plot_kwargs=dict(nxbins=15,
                   xlabel=to_latex('Xoff', True), ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, 
                    ylabel_size=28, tick_size=20))

scatter_binning(np.log10(cat['Xoff']), cat['Acc_Rate_1*Tdyn'],ax=axes[1], nxbins=15,
                   xlabel=to_latex('Xoff',True), ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, 
                ylabel_size=28, tick_size=20, legend_label='all halos',legend_size=18)

binning3d_mass(cat, axes[2], scatter_binning, 'Xoff', 'Acc_Rate_Inst', 
                   mods=[lambda x: np.log10(x), lambda x: x],
                   plot_kwargs=dict(nxbins=15,
                   xlabel=to_latex('Xoff', True), ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, 
                    ylabel_size=28, tick_size=20))

scatter_binning(np.log10(cat['Xoff']), cat['Acc_Rate_Inst'], ax=axes[3], nxbins=15,
xlabel=to_latex('Xoff',True), ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, 
                ylabel_size=28, tick_size=20)


binning3d_mass(cat, axes[4], scatter_binning, 'T/|U|', 'Acc_Rate_1*Tdyn', 
                   mods=[lambda x: np.log10(x), lambda x: x],
                   plot_kwargs=dict(nxbins=15,
                   xlabel=to_latex('T/|U|', True), ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, 
                    ylabel_size=28, tick_size=20))

scatter_binning(np.log10(cat['T/|U|']), cat['Acc_Rate_1*Tdyn'], ax=axes[5], nxbins=15,
xlabel=to_latex('T/|U|',True), ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, 
                ylabel_size=28, tick_size=20)

binning3d_mass(cat, axes[6], scatter_binning, 'T/|U|', 'Acc_Rate_Inst', 
                   mods=[lambda x: np.log10(x), lambda x: x],
                   plot_kwargs=dict(nxbins=15,
                   xlabel=to_latex('T/|U|', True), ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, 
                    ylabel_size=28, tick_size=20))

scatter_binning(np.log10(cat['T/|U|']), cat['Acc_Rate_Inst'], ax=axes[7], nxbins=15,
xlabel=to_latex('T/|U|',True), ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, 
                ylabel_size=28, tick_size=20)


binning3d_mass(cat, axes[8], scatter_binning, 'Voff', 'Acc_Rate_1*Tdyn', 
                   mods=[lambda x: np.log10(x), lambda x: x],
                   plot_kwargs=dict(nxbins=15,
                   xlabel=to_latex('Voff', True), ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, 
                    ylabel_size=28, tick_size=20))

scatter_binning(np.log10(cat['Voff']), cat['Acc_Rate_1*Tdyn'], ax=axes[9], nxbins=15,
xlabel=to_latex('Voff',True), ylabel=to_latex('Acc_Rate_1*Tdyn'), xlabel_size=22, 
                ylabel_size=28, tick_size=20)

binning3d_mass(cat, axes[10], scatter_binning, 'Voff', 'Acc_Rate_Inst', 
                   mods=[lambda x: np.log10(x), lambda x: x],
                   plot_kwargs=dict(nxbins=15,
                   xlabel=to_latex('Voff', True), ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, 
                    ylabel_size=28, tick_size=20))

scatter_binning(np.log10(cat['Voff']), cat['Acc_Rate_Inst'], ax=axes[11], nxbins=15,
xlabel=to_latex('Voff',True), ylabel=to_latex('Acc_Rate_Inst'), xlabel_size=22, 
                ylabel_size=28, tick_size=20)


plt.tight_layout()

In [ ]:
# conclusion: eta is much more sensitive to recent accretion 
# should look at this for different mass ranges (decades)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(20,10))
axes = axs.flatten()

scatter_binning( cat['scale_of_last_MM'], np.log10(cat['Xoff']),ax=axes[0], 
                nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('Xoff',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True,
                legend_label='all galaxies')

scatter_binning( cat[relaxed]['scale_of_last_MM'], np.log10(cat[relaxed]['Xoff']),ax=axes[0], 
                nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('Xoff',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True, color='blue',
                legend_label='relaxed')

scatter_binning( cat['scale_of_last_MM'], np.log10(cat['Voff']),ax=axes[1], 
                nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('Voff',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True,
                legend_label='all galaxies')

scatter_binning( cat[relaxed]['scale_of_last_MM'], np.log10(cat[relaxed]['Voff']),ax=axes[1], 
                nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('Voff',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True, color='blue',
                legend_label='relaxed')

scatter_binning( cat['scale_of_last_MM'], np.log10(cat['T/|U|']),ax=axes[2], 
                nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('T/|U|',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True,
                legend_label='all galaxies')

scatter_binning( cat[relaxed]['scale_of_last_MM'], np.log10(cat[relaxed]['T/|U|']),ax=axes[2], 
                nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('T/|U|',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True, color='blue',
                legend_label='relaxed')

# scatter_binning( cat['scale_of_last_MM'], np.log10(cat['Xoff']),ax=axes[0], 
#                 nxbins=10, xlabel=to_latex('scale_of_last_MM'), ylabel=to_latex('Xoff',True))

# scatter_binning( cat['scale_of_last_MM'], np.log10(cat['T/|U|']),ax=axes[1], nxbins=10)
plt.tight_layout()

# Plot with shapes 

In [ ]:
#hypothesis: T/U related to shape. 

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(20,10))
axes = axs.flatten()

scatter_binning( cat['q'], np.log10(cat['Xoff']),ax=axes[0], 
                nxbins=10, xlabel=to_latex('q'), ylabel=to_latex('Xoff',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True,
                legend_label='all galaxies')

scatter_binning( cat['q'], np.log10(cat['Voff']),ax=axes[1], 
                nxbins=10, xlabel=to_latex('q'), ylabel=to_latex('Voff',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True,
                legend_label='all galaxies')


scatter_binning( cat['q'], np.log10(cat['T/|U|']),ax=axes[2], 
                nxbins=10, xlabel=to_latex('q'), ylabel=to_latex('T/|U|',True),
                xlabel_size=28, ylabel_size=28, tick_size=22, no_bars=True, show_bands=True,
                legend_label='all galaxies')

plt.tight_layout()

In [ ]:
#ToDo: 
#add calculating the substructure fraction. 
#so at different time scales the universse is denser so different things might relax quicker, 
#look only at most massive progenitor. 
#vrms as measure of relaxedness 